In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import json
import pickle
from datetime import datetime
from datetime import timezone
from scipy.stats import t
from mat4py import loadmat
from distfit import distfit

In [2]:
powerdata_path = r"./data/siemens/hourly_avg_wind_speed_and_power.mat"
pricedata_path = r"./data/siemens/da_price_data_dk1_10AM.csv"

powerdata_dict_list = loadmat(powerdata_path, meta=False)
pricedata = pd.read_csv(pricedata_path)
pricedata.dropna(inplace=True)
pricedata['error'] = pricedata['forecast'] - pricedata['nominal']

# Get data out of singleton lists
powerdata_dict = {}
for key in powerdata_dict_list.keys():
    powerdata_dict[key] = [l[0] for l in powerdata_dict_list[key]]
powerdata = pd.DataFrame(powerdata_dict)

# Features

In [3]:

df_DK2_2019 = pd.read_csv('./data/ENTSOE_Generation Forecasts for Wind and Solar_DK2_201901010000-202001010000.csv')
df_DK2_2020 = pd.read_csv('./data/ENTSOE_Generation Forecasts for Wind and Solar_DK2_202001010000-202101010000.csv')
df_DK2_2021 = pd.read_csv('./data/ENTSOE_Generation Forecasts for Wind and Solar_DK2_202101010000-202201010000.csv')

df_DK1_2019 = pd.read_csv('./data/ENTSOE_Generation Forecasts for Wind and Solar_DK1_201901010000-202001010000.csv')
df_DK1_2020 = pd.read_csv('./data/ENTSOE_Generation Forecasts for Wind and Solar_DK1_202001010000-202101010000.csv')
df_DK1_2021 = pd.read_csv('./data/ENTSOE_Generation Forecasts for Wind and Solar_DK1_202101010000-202201010000.csv')

df_r_2019 = pd.read_csv('./data/ENTSOE_Actual Generation Roedsand 2_201901010000-202001020000.csv')
df_r_2020 = pd.read_csv('./data/ENTSOE_Actual Generation Roedsand 2_202001010000-202101020000.csv')
df_r_2021 = pd.read_csv('./data/ENTSOE_Actual Generation Roedsand 2_202101010000-202201020000.csv')

In [4]:
dk1_list = [df_DK1_2019, df_DK1_2020, df_DK1_2021]
df_DK1 = pd.concat(dk1_list).reset_index()
dk2_list = [df_DK2_2019, df_DK2_2020, df_DK2_2021]
df_DK2 = pd.concat(dk2_list).reset_index()
df_r_list = [df_r_2019.dropna(subset=['MTU'], how='all')[:8760], df_r_2020.dropna(subset=['MTU'], how='all')[:(8760+24)], df_r_2021.dropna(subset=['MTU'], how='all')[:8760]]
df_r = pd.concat(df_r_list).reset_index()

df_DK1.set_index("MTU (CET/CEST)", inplace=True)
df_DK1 = df_DK1.loc[~df_DK1.index.duplicated(), :]

df_DK2.set_index("MTU (CET/CEST)", inplace=True)
df_DK2 = df_DK2.loc[~df_DK2.index.duplicated(), :]


In [ ]:
realized = df_r['Generation [MW]'].to_numpy().astype(float)
realized = realized / realized.max()
# Generate production forecast values
t_samples_power = t.rvs(3.3465, loc=-11.0358, scale=342.945, size=len(realized))

# Scale from 3 GW to 10 MW
t_samples_power *= np.max(realized)/3000

forecast = realized + t_samples_power

In [ ]:

df_features = pd.DataFrame()
df_features['realized'] = realized
df_features['forecast'] = forecast
df_features['offshore_DK2'] = df_DK2['Generation - Wind Offshore  [MW] Day Ahead/ BZN|DK2'].to_numpy().astype(float)
df_features['offshore_DK1'] = df_DK1['Generation - Wind Offshore  [MW] Day Ahead/ BZN|DK1'].to_numpy().astype(float)
df_features['onshore_DK2'] = df_DK2['Generation - Wind Onshore  [MW] Day Ahead/ BZN|DK2'].to_numpy().astype(float)
df_features['onshore_DK1'] = df_DK1['Generation - Wind Onshore  [MW] Day Ahead/ BZN|DK1'].to_numpy().astype(float)
df_features['solar_DK2'] = df_DK2['Generation - Solar  [MW] Day Ahead/ BZN|DK2'].to_numpy().astype(float)

df_features.head()

In [ ]:
#--------Normalize for stability
for i in range(df_features.shape[1]-1):
    df_features.iloc[:, i] = df_features.iloc[:, i] / df_features.iloc[:, i].max()

df_features.head()

In [31]:
#----------N/A
df_features.fillna(method="ffill", inplace=True)
df_features.isnull().values.any()

False

In [32]:
#-----------Final check
df_features.shape

(26304, 7)

In [23]:
SAVE_AS_CSV = True
if (SAVE_AS_CSV):
    df_features.to_csv('./data/features.csv')

# Prices

In [12]:
df_price_2019 = pd.read_csv('./data/ENTSOE_Day-ahead Prices_201901010000-202001010000.csv')
df_price_2020 = pd.read_csv('./data/ENTSOE_Day-ahead Prices_202001010000-202101010000.csv')
df_price_2021 = pd.read_csv('./data/ENTSOE_Day-ahead Prices_202101010000-202201010000.csv')
df_price_2019.dropna(subset=['Day-ahead Price [EUR/MWh]'], inplace=True)
df_price_2020.dropna(subset=['Day-ahead Price [EUR/MWh]'], inplace=True)
df_price_2021.dropna(subset=['Day-ahead Price [EUR/MWh]'], inplace=True)
df_price_2019.reset_index(inplace=True)
df_price_2020.reset_index(inplace=True)
df_price_2021.reset_index(inplace=True)


df_price = pd.DataFrame()
df_price["Forward"] = pd.concat([df_price_2019['Day-ahead Price [EUR/MWh]'], df_price_2020['Day-ahead Price [EUR/MWh]'], df_price_2021['Day-ahead Price [EUR/MWh]']])


df_price.reset_index(inplace=True, drop=True)
df_price.drop([24818], inplace=True)
df_price.loc[len(df_price.index)+1] = [46.6]
df_price.reset_index(inplace=True, drop=True)
df_price.to_csv('./data/forward.csv')

In [13]:
df_imbalance_2019 = pd.read_csv('./data/ENTSOE_Imbalance_201901010000-202001010000.csv')
df_imbalance_2020 = pd.read_csv('./data/ENTSOE_Imbalance_202001010000-202101010000.csv')
df_imbalance_2021_all = pd.read_csv('./data/remaining_balancing_data.csv', delimiter=";", decimal=",")

df_imbalance_2021_all['HourUTC'] = pd.to_datetime(df_imbalance_2021_all['HourUTC'], format='%Y-%m-%d %H:%M')
df_imbalance_2021_all.set_index("HourUTC", inplace=True)

#--------fix based on analysis
df_imbalance_2019["Generation / + Imbalance price [EUR/MWh] - MBA|DK2"][6826] = 30.43
df_imbalance_2019["Generation / - Imbalance price [EUR/MWh] - MBA|DK2"][6826] = 35.24
df_imbalance_2019.dropna(subset=['Generation / + Imbalance price [EUR/MWh] - MBA|DK2'], inplace=True)

df_imbalance_2020.dropna(subset=['Generation / + Imbalance price [EUR/MWh] - MBA|DK2'], inplace=True)

df_imbalance_2021_all = pd.concat([df_imbalance_2021_all, pd.Series(np.nan)])
df_imbalance_2021_all["BalancingPowerPriceDownEUR"][-1] = 46.60
df_imbalance_2021_all["BalancingPowerPriceUpEUR"][-1] = 46.60

df_imbalance_dw = pd.concat([df_imbalance_2019["Generation / + Imbalance price [EUR/MWh] - MBA|DK2"], df_imbalance_2020["Generation / + Imbalance price [EUR/MWh] - MBA|DK2"], df_imbalance_2021_all["BalancingPowerPriceDownEUR"]]).reset_index()
df_imbalance_up = pd.concat([df_imbalance_2019["Generation / - Imbalance price [EUR/MWh] - MBA|DK2"], df_imbalance_2020["Generation / - Imbalance price [EUR/MWh] - MBA|DK2"], df_imbalance_2021_all["BalancingPowerPriceUpEUR"]]).reset_index()

df_imbalance = pd.DataFrame({"Up": df_imbalance_up[0].to_numpy(), "Down": df_imbalance_dw[0].to_numpy(), "Forward": df_price["Forward"]})

df_imbalance['SinglePrice'] = np.where(df_imbalance['Up'] != df_imbalance['Forward'], df_imbalance['Up'], df_imbalance['Down'])

df_imbalance.head()

df_imbalance.to_csv('./data/prices.csv')

In [14]:
#-----------Generate forecast values
t_samples_price = t.rvs(1.44887, loc=0.389, scale=12.089, size=len(df_imbalance["Forward"].to_numpy()))
t_samples_price *= 1/871.0*254.44

forecast = df_imbalance["Forward"].to_numpy() + t_samples_price

In [62]:
df_imbalance.iloc[17638:17648, :]

df_features
# ["forward_RE", "forward_FC", "UP", "DW", "production_RE", "production_FC", "Offshore DK2", "Offshore DK1", "Onshore DK2", "Onshore DK1"]
df_all = pd.DataFrame()
df_all["forward_RE"] = df_imbalance[["Forward"]]
df_all["forward_FC"] = forecast
df_all["UP"] = df_imbalance[["Up"]]
df_all["DW"] = df_imbalance[["Down"]]
df_all["production_RE"] = df_features[["realized"]]
df_all["production_FC"] = df_features[["forecast"]]
df_all["Offshore DK2"] = df_features[["offshore_DK2"]]
df_all["Offshore DK1"] = df_features[["offshore_DK1"]]
df_all["Onshore DK2"] = df_features[["onshore_DK2"]]
df_all["Onshore DK1"] = df_features[["onshore_DK1"]]

df_all.to_csv("2020_data")